# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [104]:
!pip3 install -U imbalanced-learn

In [105]:
!pip3 install --upgrade pip
!pip3 install -U numpy
!pip3 install -U scikit-learn
!pip3 install -U scipy

In [107]:
# import libraries
import pandas as pd
import numpy as np
import time

from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
#from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [ ]:
!curl -O https://raw.githubusercontent.com/udacity/workspaces-student-support/master/jupyter/workspace_utils.py

from workspace_utils import active_session    

In [70]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [47]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = df = pd.read_sql_table('CategorizedMessage', engine)
df.head()
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [71]:
def tokenize(text):
    stop_words = set(stopwords.words('english'))
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    filtered_tokens = [w for w in tokens if not w.lower() in stop_words]
 
    clean_tokens = []
    for tok in filtered_tokens:
        clean_tokens.append(lemmatizer.lemmatize(tok).lower().strip())

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [72]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

t = time.time()
pipeline.fit(X_train, y_train)
print('fittime: ', time.time() - t)

fittime:  85.96089911460876


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [74]:
y_pred = pipeline.predict(X_test)

In [75]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [76]:
np.array(y_test['related'])

array([1, 1, 1, ..., 0, 0, 1])

In [77]:
y_test.shape

(6554, 36)

In [78]:
y_pred.shape

(6554, 36)

In [79]:
print(classification_report(y_test, y_pred, target_names=y_test.columns))


                        precision    recall  f1-score   support

               related       0.84      0.93      0.88      5027
               request       0.79      0.39      0.52      1085
                 offer       0.00      0.00      0.00        26
           aid_related       0.73      0.58      0.65      2656
          medical_help       0.51      0.07      0.13       512
      medical_products       0.66      0.06      0.11       326
     search_and_rescue       0.40      0.01      0.02       186
              security       0.00      0.00      0.00       116
              military       0.52      0.07      0.12       222
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.27      0.41       409
                  food       0.82      0.45      0.58       720
               shelter       0.77      0.29      0.42       540
              clothing       0.71      0.05      0.09       106
                 money       0.56      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [80]:
## params
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fe90c603730>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [81]:
parameters =  {
     'clf__estimator__n_estimators':  [10, 50, 100],
    'clf__estimator__min_samples_split': [2, 3],
}


cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, cv=2, verbose=3)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [82]:
t = time.time()

with active_session():
    cv.fit(X_train, y_train)
    
print('time: ', time.time() - t)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.23039365273115656, total=  38.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   48.0s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.2191028379615502, total=  38.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  1.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.24259993896856882, total= 2.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.23608991964194895, total= 2.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, score=0.24585494863187876, total= 4.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, score=0.2380225816295392, total= 4.3min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10, score=0.21849252364967958, total=  35.8s
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_spl

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 30.9min finished


time:  2487.5406210422516


In [83]:
y_pred = cv.predict(X_test)

In [84]:
print(classification_report(y_test, y_pred, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5027
               request       0.87      0.49      0.63      1085
                 offer       0.00      0.00      0.00        26
           aid_related       0.74      0.70      0.72      2656
          medical_help       0.65      0.08      0.14       512
      medical_products       0.79      0.07      0.12       326
     search_and_rescue       0.72      0.07      0.13       186
              security       0.00      0.00      0.00       116
              military       0.72      0.08      0.15       222
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.37      0.52       409
                  food       0.86      0.57      0.69       720
               shelter       0.81      0.31      0.45       540
              clothing       0.71      0.09      0.17       106
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [85]:
#best params
cv.best_params_

{'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 100}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [108]:
# pipeline2 = Pipeline([
#     ('vect', CountVectorizer(tokenizer=tokenize)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', MultiOutputClassifier(BalancedRandomForestClassifier()))
# ])


# pipeline2.get_params()

### 9. Export your model as a pickle file

In [86]:
import pickle
with open('model_pkl', 'wb') as files:
    pickle.dump(cv, files)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.